In [1]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
import pickle
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV


xgboost_data = pickle.load(open('XGBoost_clean_data_all.p', 'rb'))
xgbc = XGBClassifier()


In [2]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorical_columns = xgboost_data.select_dtypes(exclude=numerics)

### Categorical Encoders

In [3]:
categorical_encoders = dict()
for col in categorical_columns:
    categorical_encoders[col] = LabelEncoder()
    label_encoder = categorical_encoders[col].fit(categorical_columns[col])
    xgboost_data[col] = categorical_encoders[col].transform(categorical_columns[col])

In [11]:
try:
    xgboost_data = xgboost_data.drop('Id', axis=1)
except KeyError:
    print('Id already removed')
X = xgboost_data.drop(['is_bad'], axis=1)
y = xgboost_data.filter(['is_bad'])

y_trans = np.asarray(y).reshape(len(y),)

Id already removed


precision: What proportion of positive identifications were actually correct?

recall: What proportion of actual positives were identified correctly?

In [12]:
xgbc.fit(X=X, y=y_trans, eval_metric='logloss')

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=20, min_child_weight=1, missing=None, n_estimators=40,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=50, seed=None,
       silent=True, subsample=1)

In [15]:
y_pred = xgbc.predict(X)
xgbc.predict_proba(X)


NameError: name 'precision_recall_fscore_support' is not defined

In [7]:
model = XGBClassifier()
scale_pos_weight = range(1, 100, 20)
param_grid = dict(scale_pos_weight=scale_pos_weight)
kfold = StratifiedKFold(n_splits=6, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, np.asarray(y).reshape(len(y),))

In [8]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))


Best: -1.058533 using {'scale_pos_weight': 20}
-1.202533 (0.001539) with: {'scale_pos_weight': 0}
-1.058533 (0.007786) with: {'scale_pos_weight': 20}
-1.443142 (0.009796) with: {'scale_pos_weight': 40}
-1.699615 (0.012679) with: {'scale_pos_weight': 60}
-1.881626 (0.011964) with: {'scale_pos_weight': 80}


In [ ]:
from matplotlib import pyplot
pyplot.errorbar(n_estimators, means, yerr=stds)
pyplot.title("XGBoost n_estimators vs Log Loss")
pyplot.xlabel('n_estimators')
pyplot.ylabel('Log Loss')
pyplot.savefig('n_estimators.png')
pyplot.show()

In [6]:
xgbc = XGBClassifier(n_estimators=40, max_depth=20, scale_pos_weight=50)

for train_index, test_index in kf.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    xgbc.fit(X_train, np.asarray(y_train).reshape(len(y_train),))
    
    y_pred = list(xgbc.predict(X_test))
    print(sum(y_pred))
    y_true = [x[0] for x in y_test.values]
    print(sum(y_true))
    print('Precision: {}'.format(precision_recall_fscore_support(y_pred=y_pred, y_true=y_true)[0]))
    print('Recall: {}'.format(precision_recall_fscore_support(y_pred=y_pred, y_true=y_true)[1]))
    print('Accuracy: {}'.format(np.sum(np.asarray(y_pred) == np.asarray(y_true)) / len(y_true)))
    print()
    
    

NameError: name 'kf' is not defined

In [ ]:
np.sum(np.asarray(y_pred) == np.asarray(y_true)) / len(y_true)